Exploratory data analysis

In [7]:
import pandas as pd

spam_data = pd.read_csv(r"C:\Users\Ashik\Downloads\spam.csv", encoding="latin-1")
spam_data.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [8]:
# Changing columns name and change target values

spam_data.rename(columns = {'v1': 'target',
                            'v2': 'text'},
                inplace = True)

spam_data.head()

,target,text,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [9]:
spam_data = spam_data.drop(['Unnamed: 2', 'Unnamed: 3','Unnamed: 2','Unnamed: 4'], axis=1)

In [10]:
spam_data['target'] = spam_data['target'].map({'ham': 0,
                                           'spam': 1})

spam_data.head()

,target,text
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [11]:
print(spam_data.iloc[0]['text'])

Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...


Data Preprocessing

In [12]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Ashik\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [13]:
# 1. Remove punctuations
spam_data['text'] = spam_data['text'].apply(lambda x: re.sub(r'[^\w\s]', '', x))

# 2. Convert to lowercase
spam_data['text'] = spam_data['text'].apply(lambda x: x.lower())

# 3. Remove stopwords
stop_words = set(stopwords.words('english'))

spam_data['text'] = spam_data['text'].apply(lambda x: ' '.join([word for word in x.split() if word.lower() not in stop_words]))

# 4. Remove numbers
spam_data['text'] = spam_data['text'].apply(lambda x: re.sub(r'\d+', '', x))

# 5. Remove words less than 2 letters
spam_data['text'] = spam_data['text'].apply(lambda x: ' '.join([word for word in x.split() if len(word) > 2]))

In [14]:
spam_data.head()

,target,text
0,0,jurong point crazy available bugis great world...
1,0,lar joking wif oni
2,1,free entry wkly comp win cup final tkts may te...
3,0,dun say early hor already say
4,0,nah dont think goes usf lives around though


Implementing ml algorithm

In [15]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [16]:
X = spam_data['text']
y = spam_data['target']
ran_state = 2023
t_size = 0.2


cv = CountVectorizer()
X = cv.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=t_size,
                                                    random_state=ran_state,
                                                    stratify = y) # To make sure "y" target is evenly distributed
                                                                  # Across training and testing data

Logistic regression model

In [17]:
# 1. Train the model
lr_model = LogisticRegression()

lr_model.fit(X_train, y_train)

# 2. Make prediction
y_lr_model_predict = lr_model.predict(X_test)

In [18]:
# Evaluate the model
print('Logistic Regression:')
print('Accuracy:', accuracy_score(y_test, y_lr_model_predict))
print('Precision:', precision_score(y_test, y_lr_model_predict))
print('Recall:', recall_score(y_test, y_lr_model_predict))
print('F1 score:', f1_score(y_test, y_lr_model_predict))

Logistic Regression:
Accuracy: 0.979372197309417
Precision: 0.9846153846153847
Recall: 0.8590604026845637
F1 score: 0.9175627240143369


Linear SVM model

In [19]:
svm_model = LinearSVC()
svm_model.fit(X_train, y_train)
y_svm_model_predict = svm_model.predict(X_test)

In [20]:
# Evaluate the model
print('Linear SVM:')
print('Accuracy:', accuracy_score(y_test, y_svm_model_predict))
print('Precision:', precision_score(y_test, y_svm_model_predict))
print('Recall:', recall_score(y_test, y_svm_model_predict))
print('F1 score:', f1_score(y_test, y_svm_model_predict))

Linear SVM:
Accuracy: 0.9820627802690582
Precision: 0.9849624060150376
Recall: 0.8791946308724832
F1 score: 0.9290780141843972


Conclusion-
The Linear SVM model has the best performance compared to Logistic Regression model

In [24]:
import os

model_folder = "./models/"

# Create the directory if it doesn't exist
os.makedirs(model_folder, exist_ok=True)

# Rest of your code
import joblib

model_file_name = "svm_best_model.joblib"

joblib.dump(svm_model, model_folder + model_file_name)


['./models/svm_best_model.joblib']

In [25]:
# Load the model
loaded_svm_model = joblib.load(open(model_folder+''+model_file_name, 'rb'))

def make_prediction(input_text):

    preprocessed_text = cv.transform([input_text])

    prediction = loaded_svm_model.predict(preprocessed_text)

    info = ''

    if(prediction[0] == 0):
        info = "Ham ✅"

    else:
        info = "Spam 🚨"

    final_info = "This message is a : {}".format(info)

    return final_info



text_example = "Thank you for your message. You have won $1000000000 send your bank information asap!!!"

print(make_prediction(text_example))

This message is a : Spam 🚨


User Interface using gradio

In [27]:
#!pip3 install gradio
import gradio as gr

C:\Users\Ashik\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [28]:
headline = "Spam Detector"

iface = gr.Interface(fn=make_prediction, inputs="text", outputs="text", title = headline)
iface.launch(share=True)

Running on local URL:  http://127.0.0.1:7860

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.
